### Prompts

In [ ]:
SYSTEM_PROMPT = """
You are a banking transaction explanation assistant.

Rules:
- Explain ONLY using the provided facts.
- Do NOT add new information.
- Do NOT speculate about user intent.
- Do NOT provide financial advice.
- Keep explanations factual and concise.
- Maximum 3 sentences.
"""

USER_PROMPT_TEMPLATE = """
Explain the following transaction insight in clear, human-friendly language.

Insight Type: {insight_type}
Severity: {severity}
Facts:
{facts}

Explanation:
"""

### Explainer

In [8]:
import json
import requests

In [9]:

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3.1:8b"

INSIGHT_FILE = "../data/transaction_insights.json"
OUTPUT_FILE = "../data/transaction_explanations.json"


In [10]:
def generate_explanation(insight):
    facts_text = "\n".join(
        [f"- {k}: {v}" for k, v in insight["facts"].items()]
    )

    prompt = USER_PROMPT_TEMPLATE.format(
        insight_type=insight["insight_type"],
        severity=insight["severity"],
        facts=facts_text
    )

    payload = {
        "model": MODEL_NAME,
        "prompt": f"{SYSTEM_PROMPT}\n{prompt}",
        "stream": False,
        "options": {
            "temperature": 0.2,
            "num_predict": 120
        }
    }

    response = requests.post(OLLAMA_URL, json=payload)
    response.raise_for_status()

    return response.json()["response"].strip()

In [11]:
with open(INSIGHT_FILE) as f:
    insights = json.load(f)

explanations = []

for insight in insights:
    explanation = generate_explanation(insight)

    explanations.append({
        "transaction_id": insight["transaction_id"],
        "insight_type": insight["insight_type"],
        "explanation": explanation
    })

with open(OUTPUT_FILE, "w") as f:
    json.dump(explanations, f, indent=2)

print(f"Local LLaMA explanations generated → {OUTPUT_FILE}")

Local LLaMA explanations generated → ../data/transaction_explanations.json
